### <center> Introducción del proyecto

Actualmente, el agua ya no es sólo un recurso natural que los humanos conumen día a día, sino que se ha convertido en un indicador importante a la hora de hacer análisis sobre la economía y el medio ambiente.

El proyecto está elaborado basando en los datos aportados por el Agua de Barcelona, y tiene como los objetivos principales detectar anomalias en los consumos de agua ,entender el costumbre de uso de agua de los ciutadanos en la provincia de Barcelona y predecir consumos futuros.

Para poder completar los objetivos indicados, se ha utilizado Python para el análisis de datos y la predicción. Además, se ha construido con echarts y apache opensourceuna una [página web](https://l912657457.github.io/AiguaDeBarcelona_DataChallenge/) en que los usuarios pueden encontrar más gráficos y visualizar los resultados obtenidos. 
Finalmente, se ha utilizado datos adicionales encontrados en INE y Idescat para relacionar el consumo de agua con otros aspectos sociales e económicos tales como la población y el turismo.

El proyecto consta de cuatro partes: 
1. Planteamiento de preguntas
2. Análisis de los contadores anormales y consumo en caso general
3. Predicción de consumos
4. Conclusión y sugerimientos


Ejecución del todo el notebook:3h

# <center>Planteamiento de preguntas

Antes de empezar a trabajar con el proyecto, se ha planteado previamente unas preguntas que serán resueldas a medida que se va analizando los datos:

1. ¿Cómo se comportan las anomalias en los consumos registrados?
2. ¿Porqué surgen las anomalias?
3. ¿Cómo afecta la pandemia en los consumos de agua?
4. ¿Existe la relación entre el turismo y el consumo de agua?

## Preprocesamiento de datos

Los bases de datos que son usados pera el análisis son de tipo de consumos y de contadores por zona, los datasets descargados de la web se guardan en 3 carpetas diferentes(comercial,domestico,industrial) según el ámbito del uso.

En primer lugar se clasifican los consumos de cada municipio en dos tipos : normal y anormal utilizando una función siseñada, en la cual se utiliza el método IQR para detectar los consumos anómalos, y se guardan los resultados en formado csv.

In [ ]:
!pip install jupyter_dash
!pip install plotly

In [10]:
# Se importan los paquetes necesarios
import pandas as pd
import numpy as np
import re
import glob
import pathlib
from pandas import isnull
import pandas as pd
import plotly.graph_objects as go  
from plotly.subplots import make_subplots
import plotly.express as px
import os

In [7]:

import re
import glob
import pathlib
def normal(global_dato):## ruta donde se guarda el conjunto de base de datos
    path=str((pathlib.Path(global_dato[0]).parent.resolve()))+"\\"
    for i in range(len(global_dato)):
        origin=pd.read_csv(global_dato[i])
        origin_pos=origin.loc[origin.CONSUMO>=0,:]
        Q1=np.percentile(origin_pos["CONSUMO"],25)
        Q3=np.percentile(origin_pos["CONSUMO"],75)
        step=1.5*(Q3-Q1)
        max_Q=Q3+step
        origin_normal=origin_pos.loc[origin.CONSUMO<max_Q,:]
        origin_anormal=origin.loc[(origin.CONSUMO<0)|(origin.CONSUMO>=max_Q),:]
        if("\\" in global_dato[i]):
            barra=([m.start() for m in re.finditer("\\\\",global_dato[i])][-1])+1
            nom=path+(global_dato[i][barra:-4])+"_normal.csv"
            nom2=path+(global_dato[i][barra:-4])+"_anormal.csv"
        else:
            nom=path+(global_dato[i][:-4])+"_normal.csv"
            nom2=path+(global_dato[i][:-4])+"_anormal.csv"
        origin_anormal.to_csv(nom2)
        origin_normal.to_csv(nom)



In [11]:
import shutil
import os

file=["Domestic_1","Domestic_2","Domestic_3","Domestic_4"]
for i in file:
    source_dir = i
    file_names = os.listdir(source_dir)
    target_dir = "DOMESTICO"
    for file_name in file_names:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

In [13]:
#Se clasifican los datos 

#Ejemplo:  nombre=[f for f in glob.glob(r"la ruta del dataset que quiera clasificar\*.csv(documentos en forma csv)")]
#tiempo estimado: 2 horas
industrial_glob=[f for f in glob.glob(r"INDUSTRIAL\*.csv")]
industrial_glob.sort()

comercial_glob=[f for f in glob.glob(r"COMERCIAL\*.csv")]
comercial_glob.sort()

domestico_glob=[f for f in glob.glob(r"DOMESTICO\*.csv")]
domestico_glob.sort()

normal(industrial_glob)
normal(comercial_glob)
normal(domestico_glob)



Una vez obtenidos todos los datos clasificados, se guardan separadamente los archivos en 6 carpetas diferentes(industria_norm, industria_anorm, domestico_norm, domestico_anorm,comercial_norm, comercial_anor)según el tipo de consumo (normal, anormal) y el ámbito de uso(domestico, comercial,indurtrial).

Después,se crean nuevos bases de datos que contienen consumos mensuales de agua de cada municipio según el tipo de uso, y se consigue mediante las funciones "formar_archivo" y "sum_mensual". 

La función "formar_archivo" identifica los archivos de formato csv en carpetas datas para que puedan ser procesados por la función "sum_mensual" y obtener datasets deseados.

In [93]:
import pandas as pd
import numpy as np
import glob, os




def sum_mensual(df1,nom,tipo):
    df = df1[['FECHA','CONSUMO']]
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y/%m/%d')
    df["AÑO"] = df["FECHA"].dt.year
    df["MES"] = df["FECHA"].dt.month
    df["DIAS"] = df["FECHA"].dt.day
    dct = {}
    for a in range(2019,2022):
        for m in range(1,13):
            dct[f"{a}-{m}"] = df[(df['AÑO'] == a) & (df['MES'] == m)]['CONSUMO'].sum()
    conc_df = pd.DataFrame()
    conc_df["SUM_CONSUMO"] = dct.values()
    conc_df["FECHA"] = dct.keys()
    conc_df.set_index("FECHA",inplace=True)
    if(tipo=='DOM'): conc_df.to_csv(rf".\sum_dom\{nom}.csv")  #exportar
    elif (tipo=='IND'): conc_df.to_csv(rf".\sum_ind\{nom}.csv")
    elif (tipo=='COM'): conc_df.to_csv(rf".\sum_com\{nom}.csv")
    return

In [94]:
def formar_archivos(direc,tipo):
    path = direc
    file = glob.glob(os.path.join(path, "*.csv")) #se identifica todos los archivos en formado csv
    dfdic={}# se crea un diccionario con clave= nombre de municipio y contenido= dataframe correspoeniende 
    for i in range(len(file)):
        name = os.path.basename(file[i])[:-11]
        dfdic[name] = pd.read_csv(file[i],sep=',')
    for key,value in dfdic.items():
        sum_mensual(value,key,tipo)

El dataframe se obtiene una columna llamada 'FECHA' que indica los meses de los años 2019-2021
Y una columna llamada 'SUM_CONSUMO' que indica la suma del consumo de cada mes 

In [95]:
#tiempo estimado : 5 min

dir_com = r'COMERCIAL\comercial_norm'#ruta relativa(relative path) en que se guardan los consumos comerciales normales
formar_archivos(dir_com,'COM')

dir_ind = r'INDUSTRIAL\industria_norm' #ruta relativa en que se guardan los consumos industriales normales
formar_archivos(dir_ind,'IND')

dir_dom = r'DOMESTICO\domestico_norm'#ruta relativa en que se guardan los consumos domésticos normales
formar_archivos(dir_dom,'DOM')

Finalmente, se juntan los datasets obtenidos y se crean tres nuevos bases de datos que contienen consumos mensuales de agua de todos los municipios según el tipo de uso.

In [117]:
import pandas as pd
import numpy as np
import glob, os

def juntar(dir,tipo):
    path = dir
    file = glob.glob(os.path.join(path, "*.csv"))

    ndf = pd.DataFrame()
    df = pd.read_csv(file[0],sep=',')
    ndf['FECHA'] = df['FECHA']
    name = os.path.basename(file[0])[:-4]
    ndf[name] = df['SUM_CONSUMO']
    for i in range(1,len(file)):
        df = pd.read_csv(file[i],sep=',')
        name = os.path.basename(file[i])[:-4]
        ndf[name] = df['SUM_CONSUMO']
    ndf.set_index("FECHA",inplace=True)
    if(tipo=='DOM'): ndf.to_excel("1_Consumo_domestico.xlsx")
    elif(tipo=='COM'): ndf.to_excel("2_Consumo_comercial.xlsx")
    elif(tipo=='IND'): ndf.to_excel("3_Consumo_industrial.xlsx")

In [118]:
comdir = r"COMERCIAL\sum_com"
juntar(comdir,'COM')

inddir = r"INDUSTRIAL\sum_ind"
juntar(inddir,'IND')

domdir = r"DOMESTICO\sum_dom"
juntar(domdir,'DOM')

## Análisis de los datos preprocesados

###  Análisis general del consumo de agua según el tipo de uso

Con los consumos normales por municipio, se elaboran unas gráficas que representan la evolución de la cantidad de agua consumida por cada municipio desde el año 2019 hasta el 2020.

En la [siguiente apartada de la página web]( https://l912657457.github.io/AiguaDeBarcelona_DataChallenge/templates/grafic-1.html) se encuentran más gráficas que representa la evolución de consumo de agua en los últimos años según su tipo. Las cuales, son interactivas que dará la posibilidad de consultar la cantidad total de agua consumida en cada mes del año y seleccionar los municipios que le interese. 




### Consumo domestico
La mayría de los municipios presentan una tendencia similar en el consumo del agua, desde el principio del 2019 hasta el final del 2020, ha habido un aumento pregresivo en los consumos.Dicho fenómeno no es exclusivo del uso doméstico, sino también está presentado en otros tipos de uso, y se supone un resultado del aumento de la instalación de contadores.

En general, durante el Diciembre y el Enero se suele aumentar el consumo de agua y en el mes de Agosto se disminuye. 

Además, se detecta que durante la pandemia, hubó una disminución significativa en el consumo doméstico en el Abril. Esta disminución podría ser debida al anuncio público por la sanidad en que el virus COVID-19 se puede sobrevivir en el agua, la cual cosa aumenta la tendencia de usar agua mineral enbotellada, también se sugiere la relación entre la hospitalización y la cuarentena de los pacientes de covid, se implica una disminución del consumo doméstico). 


In [20]:
## Consumo doméstico: Badalona, Barcelona, Castelltefels, Hospitelet de Llobregat
domestico_norm=pd.read_excel(r"C:\Users\23675\OneDrive\桌面\ABfinal\1_Consumo_domestico.xlsx")
municipis_dom=domestico_norm.loc[:,("FECHA","BADALONA","BARCELONA","CASTELLDEFELS","L_HOSPITALET LLOBR.")]
fig=go.Figure([go.Scatter(x = municipis_dom.FECHA, y = municipis_dom["BADALONA"], mode = "lines",name="BADALONA")])
for i in municipis_dom.columns[2:]:
    fig.add_trace(go.Scatter(x=municipis_dom.FECHA,y=municipis_dom[i],name=i))
fig.update_layout(title="Evolución de consumo de agua en diferentes municipios")
fig.add_vline(x=municipis_dom.FECHA[15],line_width=0.5,line_color="orange")## linia en el April del 2020
fig.show()





### Consumo comercial
Se muestra una significativa bajada del consumo en el Agosto, y el periodo furante el Diciembre y Febrero.

Durante pandemia, se demuestra una disminución del consumo en general. Un punto interesante para mencionar, es que los municipios con una alta densidad hotelera tales como Barcelona y l'Hospiltalet de Llobregat tuvieron menores consumos de agua que otraz zonas.


In [23]:
## Consumo comercial: Badalona, Barcelona, Castelltefels, Hospitelet de Llobregat
comercial_norm=pd.read_excel(r"C:\Users\23675\OneDrive\桌面\ABfinal\2_Consumo_comercial.xlsx")
municipis_com=comercial_norm.loc[:,("FECHA","BADALONA","BARCELONA","CASTELLDEFELS","L'HOSPITALET LLOBR.")]
fig=go.Figure([go.Scatter(x = comercial_norm.FECHA, y = municipis_com["BADALONA"], mode = "lines",name="BADALONA")])
for i in municipis_com.columns[2:]:
    fig.add_trace(go.Scatter(x=municipis_com.FECHA,y=municipis_com[i],name=i))
fig.update_layout(title="Evolución de consumo industrial de agua en diferentes municipios")
fig.add_vline(x=municipis_com.FECHA[15],line_width=0.5,line_color="orange")## linia en el April del 2020
fig.show()


### Consumo industrial
La evolución es similar a otros tipos de consumo, se suele haber una dismunición entre los meses de Diciembre y Febrero igual que el consumo comercial,y hubó una disminució en el Abril de 2020 que cuando ocurrió el conficionamiento por la pandemia. 

A través de la gráfica, se observa que en el Agosto se consume menos agua respecto en comparación con el Julio y el Septiembre. 
Sin embargo,durante el mes de Agosto del 2020, algunos municipios tales como Barcelona, Castelldefels y La llagosta experimentaron un aumento en el consumo de agua (需要看看那里industria textil optico和farmaceutico多不多)

In [21]:
## Consumo industrial: Badalona, Barcelona, Castelltefels, Hospitelet de Llobregat,La llagosta
industrial_norm=pd.read_excel(r"C:\Users\23675\OneDrive\桌面\ABfinal\3_Consumo_industrial.xlsx")
municipis_ind=industrial_norm.loc[:,("FECHA","BADALONA","BARCELONA","CASTELLDEFELS","L'HOSPITALET LLOBR.","LA LLAGOSTA")]
fig=go.Figure([go.Scatter(x = industrial_norm.FECHA, y = municipis_ind["BADALONA"], mode = "lines",name="BADALONA")])
for i in municipis_ind.columns[2:]:
    fig.add_trace(go.Scatter(x=municipis_ind.FECHA,y=municipis_ind[i],name=i))
fig.update_layout(title="Evolución de consumo industrial de agua en diferentes municipios")
fig.add_vline(x=municipis_ind.FECHA[15],line_width=0.5,line_color="orange")## linia en el April del 2020
fig.show()

### Creación de gráficas dinámicas de la web

Los siguientes códigos son para generar [las gráficas dinámicas de la web](https://junjielyu13.github.io/AiguaDeBarcelona_DataChallenge/templates/grafic-3.html) en que los usuarios pueden visualizar de forma clara la evolución de consumo de agua en las zonas.

In [24]:
Municipios_Codi = {
    'BADALONA':8015,
    'BARCELONA':8019,
    'BEGUES':8020,
    'CASTELLDEFELS':8056,
    'CERDANYOLA':8266,
    'CORNELLA':8073,
    'EL PAPIOL':8158,
    'ESPLUGUES':8077,
    'GAVA':8089,
    'LES BOTIGUES SITGES':8270,
    "L'HOSPITALET LLOBR.":8101,
    "L_HOSPITALET LLOBR.":8101,
    'LA LLAGOSTA':8105,
    'MONTCADA I REIXAC':8125,
    'MONTGAT':8126,
    'PALLEJA':8157,
    'SANT ADRIA':8194,
    'SANT BOI':8200,
    'SANT CLIMENT LLOB.':8204,
    'SANT CUGAT':8205,
    'SANT FELIU LL.':8211,
    'SANT JOAN DESPI':8217,
    'SANT JUST DESVERN':8221,
    'STA.COLOMA CERVELLO':8244,
    'STA.COLOMA GRAMENET':8245,
    'TORRELLES LLOBREGAT':8289,
    'VILADECANS':8301
}

def formar(dir):
    path = dir ##mod
    file = glob.glob(os.path.join(path, "*.csv"))
    df1 = pd.read_csv(file[0],sep=',')
    df1['MUNICIPIO'] = [os.path.basename(file[0])[:-4] for i in range(df1.shape[0])]
    df1['CODIGO_INE'] = [Municipios_Codi[os.path.basename(file[0])[:-4]] for i in range(df1.shape[0])]
    for i in range(1,len(file)):
        df = pd.read_csv(file[i],sep=',')
        name = os.path.basename(file[i])[:-4]
        df['MUNICIPIO'] = [name for j in range(df.shape[0])]
        df['CODIGO_INE'] = [Municipios_Codi[name] for j in range(df.shape[0])]
        df1 = pd.concat([df1,df])
    return df1

com_dir = r".\COMERCIAL\sum_com"
comercial = formar(com_dir) 

ind_dir = r".\INDUSTRIAL\sum_ind"
industrial = formar(ind_dir) 

dom_dir = r".\DOMESTICO\sum_dom"
domestico = formar(dom_dir) 

industrial['TIPO'] = ['INDUSTRIAL' for i in range(industrial.shape[0])]
comercial['TIPO'] = ['COMERCIAL' for i in range(comercial.shape[0])]
domestico['TIPO'] = ['DOMESTICO' for i in range(domestico.shape[0])]

res = pd.concat([domestico,industrial,comercial])
res.to_excel(r"sum_consumo_total.xlsx")

In [29]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import json

df = pd.read_excel(r"sum_consumo_total.xlsx")
file = open(r"map.geojson",encoding='utf-8')
municipios = json.load(file)
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H4('Consumo mensual'),
    html.P("Seleccionar el tipo de consumo:"),
    dcc.RadioItems(
        id='tipo', 
        options=["Doméstico", "Industrial", "Comercial"],
        value="Doméstico",
        inline=True
    ),
    
    
    dcc.Graph(id='graf')
])


fig1 = px.bar(df[df['TIPO']=='COMERCIAL'], 
    x="MUNICIPIO", y="SUM_CONSUMO",animation_frame='FECHA',color='SUM_CONSUMO')
fig1.update_xaxes(title_text=" ")
fig1.update_yaxes(title_text="Suma del consumo comercial")

fig2 = px.bar(df[df['TIPO']=='INDUSTRIAL'], 
    x="MUNICIPIO", y="SUM_CONSUMO",animation_frame='FECHA',color='SUM_CONSUMO')
fig2.update_xaxes(title_text=" ")
fig2.update_yaxes(title_text="Suma del consumo industrial")

fig3 = px.bar(df[df['TIPO']=='DOMESTICO'], 
    x="MUNICIPIO", y="SUM_CONSUMO",animation_frame='FECHA',color='SUM_CONSUMO')
fig3.update_xaxes(title_text="DOMESTICO")
fig3.update_yaxes(title_text="Suma del consumo doméstico")

fig1.show()
fig2.show()
fig3.show()


Se genera un maplot que lo puede consultar mediante el link .

In [30]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import json

df = pd.read_excel(r"sum_consumo_total.xlsx")
file = open(r"map.geojson",encoding='utf-8')
municipios = json.load(file)
op={'Doméstico':'DOMESTICO','Industrial':'INDUSTRIAL','Comercial':'COMERCIAL'}
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H4('Consumo mensual'),
    html.P("Seleccionar el tipo de consumo:"),
    dcc.RadioItems(
        id='tipo', 
        options=["Doméstico", "Industrial", "Comercial"],
        value="Doméstico",
        inline=True
    ),
    
    
    dcc.Graph(id='graf')
])

@app.callback(
    Output("graf", "figure"), 
    Input("tipo", "value"))
def display_choropleth(type):
    df_filt = df.loc[df['TIPO']==op[type]]
    fig = px.choropleth_mapbox(df_filt, geojson=municipios, featureidkey='properties.CODIGO_INE',locations='CODIGO_INE', color='SUM_CONSUMO',
                            range_color=(0, df_filt['SUM_CONSUMO'].max()),
                            hover_name='MUNICIPIO',
                            color_continuous_scale="darkmint",
                            animation_frame='FECHA',
                            animation_group='MUNICIPIO',
                            mapbox_style="carto-positron",
                            zoom=8.5, center = {"lat": 41.3568577, "lon": 2.0710342},
                            opacity=0.5,
                            labels={'SUM_CONSUMO':'Suma de consumo'}
                            )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


app.run_server()

Dash app running on http://127.0.0.1:8050/


### Descubrimiento

A partir de los análisis anteriores, se nota que existe un contraste entre los consumos domésticos y otros tipos de consumos durante el Diciembre y Febrero. La causa de dicha diferencia podría ser las vacaciones, ya que durante estos meses, se suele haber más fiestas y días de descanso que otros, por lo tanto, los consumos domésticos se aumentan  y los de otros tipos reducen.

En cuanto a la disminución general de consumo en Agosto, el turismo podría ser un factor importante. Según los datos encontrados en INE y Idescat y las gráficas de la [web](https://junjielyu13.github.io/AiguaDeBarcelona_DataChallenge/templates/grafic-4.html) ,tanto los viajeros interiores como los exteriores de la provincia de Barcelona, se incrementa significativamente en Agosto , y el cierre temporal de tiendas locales y la salida de los residentes causa una disminución general del consumo.

A demás, se detecta que todos los tipos de consumos de Badalona ha experimentado ha crecido rápidamente apartir del Mayo de 2021, este poría ser causado por el incremento de contadores en esta zona. En el siguiente apartado, se analiza la relación entre el aumento de contadores y consumos.


# Relación de consumo de agua con la instalación de contadores

Se crea a partir del archivo "NumComptadorsPerZona_.csv", un nuevo dataframe que representa la cantidad de contadores de agua instalados en cada mes en todos los municipios desde el año 2019 hasta el 2021.

In [32]:
import pandas as pd
import numpy as np

dict = {}
list_fecha = []
for a in range(2019,2022):
        for m in range(1,13):
            list_fecha.append(f"{a}-{m}")
dict['FECHA'] = list_fecha            

df = pd.read_csv(r"NumComptadorsPerZona_.csv",sep=',')# ruta en que está el archivo "NumComptadorsPerZona_.csv"
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df["AÑO"] = df["FECHA"].dt.year
df["MES"] = df["FECHA"].dt.month
lis_ciudad = np.sort(df['CIUDAD'].unique())

for ciudad in lis_ciudad:
    list_cont = []
    for a in range(2019,2022):
        for m in range(1,13):
            list_cont.append(df[(a == df['AÑO'])&(df['MES']==m)&(ciudad == df['CIUDAD'])]['NUM_CONTADOR'].sum())
    dict[ciudad]=list_cont    

ndf = pd.DataFrame(dict)

ndf.to_excel(rf"4_Contador.xlsx") # ruta en que se guarda el nuevo dataset

### Elaboración de tabla de proporción de contadores anormales 

En relación con el descubrimiento mencionado, se hará a continuación un estudio sobre la relación entre el consumo de agua y el número de contadores por zona.

El análisis consiste en tres etapas:

1. Elaboración de tablas de municipios ordenada según la porporción de contadores anormales
2. Clasificació de anomalías en contadores
3. Elaboración de tabla de correlaciones

## Elaboración de tablas de errores

Se crea una función "tabla_error" que crea un dataframe con dos columnas: zonas y prop_error. La columna "zonas" recoge todos los municipios presente en los tipos de consumo (industrial,comercial y doméstico), y la "prop_error" indica la porporción de contadores anómalos en cada municipio. 


In [33]:
#tiempo estimado: 1s
import glob
import re
from pandas import isnull

def tabla_error(d_glob,anor): ####resultados no en porcentaje, en caso necesario ,multiplicar*100
                               ### d_glob: lista de rutas de datos de consumo de agua según tipo de consumo  ; 
                               # anor: lista de rutas de datasets con sólo consumos aómalos según tipo de consumo
    zonas=[]
    for i in d_glob:
        pos=i.find(".csv")
        if("\\" in i):
            barra=([m.start() for m in re.finditer("\\\\",i)][-1])+1
            zonas.append(i[barra:pos])
        else:  
            zonas.append(i[0:pos])

    error_glob=pd.DataFrame(columns=["Zonas","prop_error"])
    error_glob["Zonas"]=zonas

    for k in range(len(d_glob)):
        glob=pd.read_csv(d_glob[k])
        anorm_glob=pd.read_csv(anor[k])
        contadores=anorm_glob.ID_CONTADOR.unique()
        pro_error=[]
        for j in range(len(contadores)):
            frequencia=(glob.loc[glob.ID_CONTADOR==contadores[j]]).shape[0] #frecuancia global de contadores
            error_freq=(anorm_glob.loc[anorm_glob.ID_CONTADOR==contadores[j]]).shape[0] #frecuencia total de contador error
            pro_error.append((error_freq)/(frequencia))

        error_glob["prop_error"][k]=sum(pro_error)/(len(glob.ID_CONTADOR))

    return(error_glob)

Se crean tablas de error según tipo de consumo:

In [34]:

#tiempo estimado: 55 min

#Tabla de error para consumo industrial
industrial_glob=[f for f in glob.glob(r".\INDUSTRIAL\*.csv")]
anor_indus=[f for f in glob.glob(r".\INDUSTRIAL\industria_anorm\*csv")]
industrial_glob.sort()
anor_indus.sort()

error_indus=tabla_error(industrial_glob,anor_indus)

#Tabla de error para consumo comercial
comercial_glob=[f for f in glob.glob(r".\COMERCIAL\*.csv")]
anor_comercial=[f for f in glob.glob(r".\COMERCIAL\comercial_anorm\*csv")]
comercial_glob.sort()
anor_comercial.sort()

error_comercial=tabla_error(comercial_glob,anor_comercial) 

#Tabla de error para consumo doméstico
domestico_glob=[f for f in glob.glob(r".\DOMESTICO\*.csv")]
anor_domestico=[f for f in glob.glob(r".\DOMESTICO\domestico_anorm\*csv")]
domestico_glob.sort()
anor_domestico.sort()

error_domestico=tabla_error(domestico_glob,anor_domestico)


In [35]:
print(error_indus.sort_values(by="prop_error",ascending=False).head(5),"\n")
print(error_comercial.sort_values(by="prop_error",ascending=False).head(5),"\n")
print(error_domestico.sort_values(by="prop_error",ascending=False).head(5))


                  Zonas prop_error
0              BADALONA   0.000482
22  STA.COLOMA CERVELLO   0.000452
10          LA LLAGOSTA   0.000403
4            CERDANYOLA    0.00035
12    MONTCADA I REIXAC   0.000346 

                  Zonas prop_error
19  STA.COLOMA GRAMENET   0.001027
0              BADALONA   0.000992
18  STA.COLOMA CERVELLO   0.000785
5              CORNELLA   0.000561
4            CERDANYOLA   0.000525 

                  Zonas prop_error
21  STA.COLOMA GRAMENET   0.000717
0              BADALONA   0.000638
20  STA.COLOMA CERVELLO   0.000464
15             SANT BOI   0.000269
5              CORNELLA   0.000169


Se hace una lista de los primeros 5 municipios con mayor porporción de contadores que registran más consumos anormales según tipo de consumo.

En relación con del consumo industrial, los primeros 5 municipios con mayor porporción de contadores anormales son :  Badalona, Santa Coloma de Cevelló, La llagosta, Cerdanyola y Montcada i Reixac. 

Para el consumo comercial, los primeros 5 son:  Santa Coloma de Gramenet, Badalona, Santa Coloma de Cervelló, Cornellà de Llobregat y Cerdanyola de Vallès.

Para el consumo doméstico, los primeros 5 son:  Santa Coloma de Gramenet, Badalona, Santa Coloma de Cervelló, Sant Boi y Cornellà de Llobregat.

Se observa que Badalona y Santa Coloma de Cervelló son zonas que aparacen en todas las tres tablas. Por lo tanto, se dijuban gráficas para ilustrar la evolución del número de contadores y la cantidad de agua consumida.

In [37]:

#Gráfica para Badalona
contador=pd.read_excel(r".\4_Contador.xlsx")
comercial=pd.read_excel(r".\2_Consumo_comercial.xlsx")
industrial=pd.read_excel(r".\3_Consumo_industrial.xlsx")
domestico= pd.read_excel(r".\1_Consumo_domestico.xlsx")
figura=go.Figure([go.Scatter(x=contador.FECHA.astype("datetime64"),y=contador["BADALONA"],name="Contador")])
figura.add_trace(go.Scatter(x=comercial.FECHA.astype("datetime64"),y=comercial["BADALONA"],name="Comercial"))
figura.add_trace(go.Scatter(x=domestico.FECHA.astype("datetime64"),y=domestico["BADALONA"],name="Domestico"))
figura.add_trace(go.Scatter(x=industrial.FECHA.astype("datetime64"),y=industrial["BADALONA"],name="Industrial"))
figura.update_layout(title="Badalona")

#Cabe la posibilidad de descatar las opciones haciendo click sobre cualquiera(s) línea(s) de color en la parte 
# derecha la gráfica


In [38]:
figura=go.Figure([go.Scatter(x=contador.FECHA.astype("datetime64"),y=contador["STA.COLOMA CERVELLO"],name="Contador")])
figura.add_trace(go.Scatter(x=comercial.FECHA.astype("datetime64"),y=comercial["STA.COLOMA CERVELLO"],name="Comercial"))
figura.add_trace(go.Scatter(x=domestico.FECHA.astype("datetime64"),y=domestico["STA.COLOMA CERVELLO"],name="Donméstico"))
figura.add_trace(go.Scatter(x=industrial.FECHA.astype("datetime64"),y=industrial["STA.COLOMA CERVELLO"],name="Industrial"))
figura.update_layout(title="STA.COLOMA CERVELLO")

A partir de las gráficas anteriores, se observa la evolución de instalación de contadores presentan tendencias similares que las de consumos, sobretodo en el doméstico y comercial.

### Tabla de correlación

Se supone que existe una relación entre el consumo mensual de agua y la instalación de contadores de agua en cada mes. Para verificar la suposición, se usa el coeficiente de Pearson para medir la intensidad de asociación entre la cantidad de agua consumida y  de contadores.

Además, también se usará datos de INE sobre la evolución de población en los municipio de Barcelona desde el año 2019 hasta el 2020 para saber mediante el coeficiente de Pearson,si puede haber una asociación significativa entre el número de habitantes y la cantidad de contadores instaladas. 



In [126]:
contador=pd.read_excel(r".\4_Contador.xlsx")
comercial=pd.read_excel(r".\2_Consumo_comercial.xlsx")
industrial=pd.read_excel(r".\3_Consumo_industrial.xlsx")
domestico= pd.read_excel(r".\1_Consumo_domestico.xlsx")
domestico.rename(columns={"L_HOSPITALET LLOBR.":"L'HOSPITALET LLOBR."},inplace=True)
contador.FECHA=contador.FECHA.astype("datetime64").dt.year
poblacion=pd.read_excel(r".\poblacion_zona.xlsx")

In [127]:
 #tabla de número total de contadores por año, 
 # se usa para calcular el coeficiente de Pearson entre el contador y la población.
cont_any=pd.DataFrame(index=["0","1","2"],columns=contador.columns[1:])
for i in cont_any.columns:
    cont_any.loc["0",i]=sum(contador.loc[contador.FECHA==2019,i])
    cont_any.loc["1",i]=sum(contador.loc[contador.FECHA==2020,i])
    cont_any.loc["2",i]=sum(contador.loc[contador.FECHA==2021,i])
cont_any.FECHA=[2019,2020,2021]

In [122]:
cont_any

,FECHA,BADALONA,BADIA,BARCELONA,BEGUES,CASTELLDEFELS,CERDANYOLA,CORNELLA,EL PAPIOL,ESPLUGUES,...,SANT CLIMENT LLOB.,SANT CUGAT,SANT FELIU LL.,SANT JOAN DESPI,SANT JUST DESVERN,STA.COLOMA CERVELLO,STA.COLOMA GRAMENET,TIANA,TORRELLES LLOBREGAT,VILADECANS
0,2019,660549,0,21432906300,23080412,122830460,299491,614495,14303441,63648864,...,5815783,52760,153081908,121833199,20833130,21333,197245,0,7611671,40761647
1,2020,1680920,44156,41620672310,26692861,208903633,661639,1260565,16003659,95760751,...,14518143,64449,175715577,132559707,38472143,53965,517082,9690,17354748,118533468
2,2021,170102300,46284,51888306510,27428472,216800181,121691277,140856142,16689584,106189099,...,15395014,61248,176446453,130786500,44914578,6406873,88778724,17748,20625302,218582498


In [128]:
#Tabla de contadores
import statistics
correlacion =pd.DataFrame(columns=poblacion.columns[2:],index=["corr_comer","corr_indus","corr_domes","poblacion"])
for i in correlacion.columns:
    if i in comercial.columns:
        correlacion.loc["corr_comer",i]=comercial[i].corr(contador[i])
    else: correlacion.loc["corr_comer",i]=0

    if i in industrial.columns:
        correlacion.loc["corr_indus",i]=industrial[i].corr(contador[i])
    else: correlacion.loc["corr_indus",i]=0

    if i in domestico.columns:
        correlacion.loc["corr_domes",i]=domestico[i].corr(contador[i])
    else: correlacion.loc["corr_domes",i]=0

    if i in poblacion.columns:
        cont_any[i]=cont_any[i].astype(str).astype("int64")
        correlacion.loc["poblacion",i]=np.corrcoef(poblacion[i],cont_any[i])
    else:
        correlacion.loc["poblacion",i]=0

pd.set_option('display.max_columns', None)
correlacion

,BADALONA,BARCELONA,BEGUES,CASTELLDEFELS,CERDANYOLA,CORNELLA,EL PAPIOL,ESPLUGUES,GAVA,LES BOTIGUES SITGES,L'HOSPITALET LLOBR.,MONTCADA I REIXAC,MONTGAT,PALLEJA,SANT ADRIA,SANT BOI,SANT CLIMENT LLOB.,SANT FELIU LL.,SANT JOAN DESPI,SANT JUST DESVERN,STA.COLOMA CERVELLO,STA.COLOMA GRAMENET,TORRELLES LLOBREGAT,VILADECANS,LA LLAGOSTA,SANT CUGAT
corr_comer,0.997264,0.887346,0.36701,0.884654,0.995229,0.993155,0.830222,0.886329,0.729479,0,0.766956,0.925908,0.957878,0.532134,0.66628,0.987393,0.880662,0.501105,0,0.875442,0.975786,0.997343,0.883224,0.967376,0,0
corr_indus,0.98239,0.700739,0.40907,0.582609,0.904556,0.929635,0.406769,0.312408,0.688098,0.336636,0.715275,0.627738,0.797786,0.095452,0.473465,0.836003,0.966117,0.169455,0.021293,0.391424,0.905682,0.575418,0.660996,0.545151,0.758002,0.372984
corr_domes,0.998162,0.950997,0.962751,0.985884,0.996842,0.992985,0.919821,0.937677,0.927457,0.732804,0.923756,0.977803,0.990834,0.75744,0.925099,0.989734,0.977337,0.769106,0.754684,0.936542,0.998583,0.997745,0.989465,0.992843,0,0
poblacion,"[[1.0, 0.4586808158507722], [0.458680815850772...","[[0.9999999999999998, 0.7624900171129245], [0....","[[1.0, 0.9987945567039208], [0.998794556703920...","[[0.9999999999999999, 0.8169973187011816], [0....","[[1.0, -0.7231765814269444], [-0.7231765814269...","[[1.0, 0.034911351129341114], [0.0349113511293...","[[1.0, 0.8977096965525972], [0.897709696552597...","[[1.0, 0.4643740474281349], [0.464374047428134...","[[1.0, 0.8482441281233715], [0.848244128123371...","[[1.0, 0.8748418758210195], [0.874841875821019...","[[1.0, 0.4376768771942771], [0.437676877194277...","[[1.0, 0.9960991788143554], [0.996099178814355...","[[1.0, 0.9905481140176009], [0.990548114017600...","[[1.0, 0.9967277678821768], [0.996727767882176...","[[1.0, 0.7767464214228637], [0.776746421422863...","[[1.0, -0.24419225161062652], [-0.244192251610...","[[1.0, 0.8966058200250789], [0.896605820025078...","[[0.9999999999999999, 0.999447611161249], [0.9...","[[1.0, 0.660536510938751], [0.660536510938751,...","[[1.0, 0.45560964017384953], [0.45560964017384...","[[0.9999999999999999, 0.5731363337276238], [0....","[[1.0, -0.44995021487361814], [-0.449950214873...","[[0.9999999999999999, 0.9125524729989264], [0....","[[0.9999999999999998, 0.08102663358566191], [0...","[[1.0, -0.42191402620454277], [-0.421914026204...","[[1.0, 0.8174137275678872], [0.817413727567887..."


### Descubrimiento

Mediante la tabla de correlaciones, se nota que ,en general, el número de contadores está positivamente asociado con el consumo comercial y doméstico. Por demás, los municipios que están altamente relacionadas con el número de contadores son los primeros que aparecen en las tablas de errores obtenidas anteriormente, así como Badalona, Santa Coloma de Cervelló y Cornellà de Llobregat.


# Tipos de anomalias detectadas en los contadores

Se construye una función llamada "contadores" que ordena a partir los archivos con consumos anormales, los contadores según su frecuencia y calcula el porcentaje de consumos negativos y positivos anormales (consumos extraordinariamente altos) que registra cada contador. 

Además, elabora una tabla en que recoge los k primeros contadores que registran más consumos anormales según si son positivos y negativos.

In [28]:
### funcion para coger contadores que quiera
import glob
import re 
from pandas import isnull
def contadores (anor,k): #anor: csv file con consumos anormales ; k=Número de contadores anormales en cada municipio ordenando según proporción 
    zonas=[]
    for i in anor:
        pos=i.find("_anormal.csv")
        if("\\" in i):
            barra=([m.start() for m in re.finditer("\\\\",i)][-1])+1
            zonas.append(i[barra:pos])
        else:  
            zonas.append(i[0:pos])
    cont=pd.DataFrame(columns=["zona","cont_neg","perc_neg","cont_pos","perc_pos"])
    cont["zona"]=zonas
    for j in range(len(anor)):
        glob_anor=pd.read_csv(anor[j])
        neg_anor=glob_anor.loc[glob_anor.CONSUMO<0,:]
        pos_anor=glob_anor.loc[glob_anor.CONSUMO>0,:]
        cont_neg=(round((neg_anor.ID_CONTADOR.value_counts(ascending=False))/(neg_anor.shape[0])*100,2)).rename_axis("contador").reset_index(name="counts")
        cont_pos=(round((pos_anor.ID_CONTADOR.value_counts(ascending=False))/(pos_anor.shape[0])*100,2)).rename_axis("contador").reset_index(name="counts")
        cont["cont_neg"][j]=np.array(cont_neg.contador[0:k,]).astype("object") #k contadores que reistran más consumos negativos
        cont["perc_neg"][j]=np.array(cont_neg.counts[0:k,]).astype("object")   #porporción de los k contadores que registran consumos negativos en la base de datos
        ##identico ,pero con consumos positivos anormales
        cont["cont_pos"][j]=np.array(cont_pos.contador[0:k,]).astype("object")
        cont["perc_pos"][j]=np.array(cont_pos.counts[0:k,]).astype("object")
    return(cont)
##Ejemplo
    #industrial=[f for f in glob.glob(r"C:\agua de barcelona\INDUSTRIAL\anormal\*csv")]
    #cont_4=contadores(industrial,4)
    

Utilizando la funcion anterior,se elaboran tablas de los primeros 4 contadores que producen mayor porporción de consumos anormales. 

Las descripciones de cada columna de la tabla son: 

Zona: municipios en los que se encuentran los contadores

cont_neg: los primeros 4 contadores que registran más consumos negativos [cont1 cont2 ...]

perc_neg: la porporción de dichos contadores negativos entre todos los contadores que registran consumos negativos

cont_pos: los primeros 4 contadores que registran más consumos positivos anormales(consumos muy altos)[cont1 cont2 ...]

perc_pos: la porporción de dichos contadores positivos entre todos los contadores que registran consumos positivos



In [27]:

#Consumo industrial
anor_indus=[f for f in glob.glob(r".\INDUSTRIAL\industria_anorm\*csv")]
anor_indus.sort()
cont_4=contadores(anor_indus,4)
#consumo domestico
anor_domestico=[f for f in glob.glob(r".\DOMESTICO\domestico_anorm\*csv")]
anor_domestico.sort()
cont_4_domes=contadores(anor_domestico,4)
#consumo comercial
anor_comercial=[f for f in glob.glob(r".\COMERCIAL\comercial_anorm\*csv")]
anor_comercial.sort()
cont_4_comer=contadores(anor_comercial,4)

#### Diagrama de dispersión de consumos registrados contadores

A partir de las tablas creadas anteriormente, se elabora un conjunto de diagramas de puntos de algunos contadores para conocer los distintos tipos de consumos anormales(negativo/positivo).

Consumos negativos

In [ ]:
#cont_4=pd.read_excel(r"C:\agua de barcelona\INDUSTRIAL\anormal\cont_4.xlsx")

In [13]:
cont_4.head(3)

,zona,cont_neg,perc_neg,cont_pos,perc_pos
0,BADALONA,"[92, 307, 42, 286]","[31.03, 28.09, 6.5, 5.03]","[382, 387, 388, 229]","[8.48, 8.47, 8.13, 6.12]"
1,BARCELONA,"[754, 2491, 425, 2966]","[14.14, 7.31, 5.58, 5.21]","[3250, 3246, 3103, 3146]","[0.47, 0.47, 0.47, 0.47]"
2,BEGUES,"[16, 26, 76, 105]","[58.37, 35.29, 5.43, 0.45]","[72, 15, 99, 91]","[14.29, 14.09, 7.04, 6.61]"


In [36]:
## primero seleccionar la base de datos que queremos analizar utilizando la tabla generada anterormente.
datas=pd.read_csv(r".\INDUSTRIAL\STA.COLOMA CERVELLO.csv")#introduce la ruta de la base de datos
datas["FECHA"]=datas["FECHA"].astype("datetime64")
datas.sort_values(by="FECHA",inplace=True)
contador=[int(i) for i in (list(cont_4.cont_neg[22]))] #cont_neg[n] n=posicion del municipio en cont_4
##definir IQR 
Q1=np.percentile(datas["CONSUMO"],25)
Q3=np.percentile(datas["CONSUMO"],75)
step=1.5*(Q3-Q1)
max_Q=Q3+step## consumo maximo, una vez pasado, se considera anormal
##crear nueva columna normalidad
datas["normalidad"]=np.where(((datas.CONSUMO>=0)&(datas.CONSUMO<max_Q)),"normal","anormal")


Consumos positivos

In [41]:
datas=pd.read_csv(r"INDUSTRIAL\STA.COLOMA CERVELLO.csv")
datas["FECHA"]=datas["FECHA"].astype("datetime64")
datas.sort_values(by="FECHA",inplace=True)
contador=[int(i) for i in (cont_4.cont_pos[22])]
##definir IQR 
Q1=np.percentile(datas["CONSUMO"],25)
Q3=np.percentile(datas["CONSUMO"],75)
step=1.5*(Q3-Q1)
max_Q=Q3+step
##crear nueva columna normalidad
datas["normalidad"]=np.where(((datas.CONSUMO>=0)&(datas.CONSUMO<max_Q)),"normal","anormal")

Gráfico para contadores con consumos negativos o positivos

In [42]:
##NO TOCAR NADA, excepto valores dentro de fig.update_layout 
import pandas as pd
import plotly.graph_objects as go  
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=len(contador), cols=1,
    subplot_titles=(contador))
d = {'normal':'orange',  "anormal":'red',}
for i in range(len(contador)):
    date=datas.loc[datas.ID_CONTADOR==contador[i],"FECHA"]
    consumo=datas.loc[datas.ID_CONTADOR==contador[i],"CONSUMO"]
    color=[d[k] for k in datas.loc[datas.ID_CONTADOR==contador[i],'normalidad'].values]
    fig.add_trace(go.Scatter(x=date,
                             y=consumo,
                             mode="markers",
                             marker_color=color,
                             showlegend=False
                        
                        
    )
    ,row=i+1,col=1
    )
fig.update_layout(
    width=1000,
    height=2000,)

fig.show()

Escogiendo como ejemplos Gava y Badalona, se hace individualmente unos gráficos de disperciones para algunos contadores, y se clasifican los consumos anormales en 3 tipos: 

- Negativo absoluto: los consumos registrados por el contador es siempre negativo
- Puntual : los consumos anormales aparecen aleatoriamente 
- Periódico: los consumos anormales aparecen periodicamente 



In [43]:
### negativo absoluto : contador 273 de Gava en uso industrial 
import plotly.express as px
data=pd.read_csv(r"INDUSTRIAL\GAVA.csv")
Q1=np.percentile(data["CONSUMO"],25)
Q3=np.percentile(data["CONSUMO"],75)
step=1.5*(Q3-Q1)
max_Q=Q3+step
data["normalidad"]=np.where(((data.CONSUMO>=0)&(data.CONSUMO<max_Q)),"normal","anormal")
color_discrete_map = {'normal': "orange", 'anormal': 'red'}
figura=px.scatter(data.loc[data.ID_CONTADOR==273],x="FECHA",y="CONSUMO",color="normalidad",title="Gava(industrial): contador 273",color_discrete_map=color_discrete_map)
figura.show()

In [44]:
### puntual: contador 42 de Badalona en uso industrial
data=pd.read_csv(r"INDUSTRIAL\BADALONA.csv")
data["normalidad"]=np.where(((data.CONSUMO>=0)&(data.CONSUMO<max_Q)),"normal","anormal")
color_discrete_map = {'normal': "orange", 'anormal': 'red'}
figura=px.scatter(data.loc[data.ID_CONTADOR==42],x="FECHA",y="CONSUMO",color="normalidad",title="Badalona (industrial): contador 42",color_discrete_map=color_discrete_map)

figura.add_vrect(x0="2021-08-13", x1="2021-08-21",
              annotation_text="conusmo:-1l/día", annotation_position="top left",
              fillcolor="LightSalmon", opacity=0.25, line_width=0)

In [45]:
### periodico : contador 317 de Gava en uso industrial
data=pd.read_csv(r"INDUSTRIAL\GAVA.csv")
data["normalidad"]=np.where(((data.CONSUMO>=0)&(data.CONSUMO<max_Q)),"normal","anormal")
color_discrete_map = {'normal': "orange", 'anormal': 'red'}
figura=px.scatter(data.loc[data.ID_CONTADOR==317],x="FECHA",y="CONSUMO",color="normalidad",title="Gava (industrial): contador 317",color_discrete_map=color_discrete_map)

figura.add_vrect(x0="2019-04-20", x1="2019-10-13",
              annotation_text="de 20-04 a 13-10", annotation_position="top left",
              fillcolor="LightSalmon", opacity=0.25, line_width=0)
figura.add_vrect(x0="2020-06-02", x1="2020-10-04",
              annotation_text="de 02-06 a 04-10", annotation_position="top left",
              fillcolor="LightSalmon", opacity=0.25, line_width=0)
figura.add_vrect(x0="2021-03-14", x1="2021-11-02",
              annotation_text="de 14-03 a 02-11", annotation_position="top left",
              fillcolor="LightSalmon", opacity=0.25, line_width=0)

In [72]:
ymd = pd.date_range('2019-01-01','2021-12-14',freq='D').strftime('%Y-%m-%d').to_list()
def sum_diario(df):
    lis = []
    df = df[['FECHA','CONSUMO']]
    for dia in ymd:
        lis.append(df[df['FECHA']==dia]['CONSUMO'].sum())
    return lis

def formar(direc,tipo):
    path = direc
    file = glob.glob(os.path.join(path, "*.csv"))
    dfdic={}
    fechas = {'FECHA':ymd}
    df = pd.DataFrame(fechas) 
    for i in range(len(file)):
        name = os.path.basename(file[i])[:-11]
        dfdic[name] = pd.read_csv(file[i],sep=',')

    for key,value in dfdic.items():
        lis = sum_diario(value)
        df[key]=lis
    df.set_index('FECHA',inplace=True)
    if tipo == 'DOM': df.to_excel(r".\sum_diario_domestic.xlsx")
    elif tipo == 'COM': df.to_excel(r".\sum_diario_comercial.xlsx")
    elif tipo == 'IND': df.to_excel(r".\sum_diario_industrial.xlsx")


dom = r".\domestico_normal"
formar(dom,'DOM')
ind = r'.\industrial_normal'
formar(ind,'IND')
com = r'.\comercial_normal'
formar(com,'COM')



# <center> Predicción de consumos 

<font face="GEORGIA">Observaciones en los datos que ayudan a la predicción: 
1. Se presenta datos estacionarios tanto por semanas, meses como por años.  
2. se observa que los eventos puntuales o duraderos durante los años previenen o fomentan el consumo de agua industrial, doméstico como comercial.  
3. Hay datos excepcionamente grandes o pequeños.  
4. **El consumo se incrementa o se disminuye concordando con el número de contadores de la zona**
5.  ***La disponibilidad de los datos se extiende solo hasta el día 14 en el mes diciembre 2021***.
6. **Presentan tres tipos de errores sobre los contadores: puntual, negativo absoluto y periódico.**


A base de lo realizado, hacemos cumplir nuestra meta de predecir los futuros consumos tanto mensuales como diarios de cada zona por actividades de naturaleza comercial, industrial y doméstico implementando en los datos procesados(sin errores) como sum_diario_comercial.xlsx, sum_mensual_comercial.xlsx y etc, con la ayuda de la Inteligencia Artificial.  
Nos llamó especialmente atención el modelo NeuralProphet; A diferencia de los modelos tradicionales como es el ejemplo de ARIMA y Prophet desarrollado por Facebook, y de los modelos inteligentes como Auto-Regression net (ARnet) desarrollado por Amazon que utiliza el Deep Learning. NeuralProphet sin embargo, a pesar de estar basado en Prophet, mejora alguna de las desventajas de este construyendo una puente entre las técnicas estadísticas con el marco de Deep Learning similar al del ARnet.


Ventajas del modelo NeuralProphet:  
    1. Soporta Auto-Regression y covariantes  
    2. Modelo híbrido (linear <-> Neural Network)  
    3. Python package basado en Pytorch. (Haciendo que sea adaptivo a differentes versiones.)
    4. User-friendly package: Cuando hay missing data, se rellena automáticamente con las técnicas como "bi-directional linear interpolation" o "central rolling average".  
    5. Presenta output de single step-ahead o multi step-ahead forecasts.  


https://arxiv.org/abs/2111.15397?fbclid=IwAR2vCkHYiy5yuPPjWXpJgAJs-uD5NkH4liORt1ch4a6X_kmpMqagGtXyez4  
https://neuralprophet.com/
</font>


## Preparación de las instalaciones del deep learning framework y el modelo:

In [ ]:
pip install neuralprophet

In [ ]:
pip install pytorch

## Hyperparameters para los casos generales de predicción.

- loss function: Lasso 
- optimizer: AdamW  
- period : 4 (meses o días)  
- Batch size and epochs: Approximated from the dataset size  
- learning rate: Approximated with a learning-rate range test

# Implementación del modelo deep learning (Require GPU para correrlo con velocidad):

In [115]:
import pandas as pd
import torch
import plotly.graph_objects as go
from neuralprophet import NeuralProphet
import os

import numpy as np
def NProphet(data_location,section2,period=4,save_prediction=False,save_img = False,save_model = False,pred_show=False,para_show=False):

    section1 = pd.read_excel(data_location)
    consumo = data_location.split("\\")[-1][:-3]
    mensual = consumo.split("_")[1] == "mensual"
    actividad = consumo.split("_")[2][:-2]
    print(actividad)
    print(mensual)
    print(consumo)
    # def get_var_name(variable):
    #     globals_dict = globals()
    #     return [var_name for var_name in globals_dict if globals_dict[var_name] is variable]
    if mensual == True:
        df = pd.DataFrame({"ds":section1.iloc[:,1],"y":section1[section2]})
        m = NeuralProphet(loss_func=torch.nn.L1Loss)
        df_train, df_test = m.split_df(df, freq="M", valid_p=0.1)
        metrics = m.fit(df_train, freq="M", validation_df=df_test)
    else:
        df = pd.DataFrame({"ds":section1["FECHA"],"y":section1[section2]})
        m = NeuralProphet(loss_func=torch.nn.L1Loss)
        print("--------------------------------")
        df_train, df_test = m.split_df(df, freq="D", valid_p=0.03)
        metrics = m.fit(df_train, freq="D",validation_df = df_test)
    cost=metrics.loc[metrics.shape[0]-1,"RMSE"]
    pred_train = m.predict(df)
    next_dataset = m.make_future_dataframe(df_test,periods=period)
    pred_next = m.predict(df=next_dataset)

    df = pd.concat([pred_train,pred_next])
    
    if save_prediction == True:
        df.to_csv(rf"C:\Users\23675\OneDrive\桌面\ABChallenge\Neuralprophet result\result_{consumo}_{section2}.csv")

    if mensual ==True:
        final = go.Figure()
        final.add_trace(go.Scatter(x=df['ds'], y=df["yhat1"], mode='lines', name='Training set'))
        final.update_layout(
        title={
        'text': f"Predicción del consumo mensual(Litro/Mes) {actividad} en {section2}",'y':0.95,'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },xaxis_title="Meses",yaxis_title="Consumo(Litro/Mes)")
        final.add_trace(go.Scatter(x=df['ds'][35:35+period+1], y=df['yhat1'][35:35+period+1],mode='lines',name='Prediction'))
        final.add_trace(go.Scatter(x=df.loc[:34,'ds'], y=df.loc[:34,'y'],mode='markers',name='Actual',line_color="firebrick"))
        fig_param = m.plot_parameters(plotting_backend="plotly")
        fig_param.update_layout(
        title={
        'text': f"Análisis de la tendencia del consumo {actividad} en {section2}",'y':1,'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
        )
    else:           
        final = go.Figure()
        final.add_trace(go.Scatter(x=df['ds'], y=df["yhat1"], mode='lines', name='Training set'))
        final.update_layout(
        title={
        'text': f"Predicción del consumo diario(Litro/Día) {actividad} en {section2}",'y':0.95,'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },xaxis_title="Meses",yaxis_title="Consumo(Litro/Día)")

        final.add_trace(go.Scatter(x=df['ds'][1078:1078+period+1], y=df['yhat1'][1078:1078+period+1],mode='lines',name='Prediction'))
        final.add_trace(go.Scatter(x=df['ds'], y=df['y'],mode='markers',name='Actual',line_color="firebrick"))
        fig_param = m.plot_parameters(plotting_backend="plotly")
        fig_param.update_layout(
        title={
        'text': f"Análisis de la tendencia del consumo {actividad} en {section2}",'y':1,'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
        )
    final.add_annotation(text=f"RMSE: {round(cost,2)}",
            xref="paper", yref="paper",
            x=0.9, y=0.1)
    if save_model == True:
        torch.save(m.model.state_dict(),rf"C:\Users\23675\OneDrive\桌面\ABChallenge\Neuralprophet result\model_{consumo}_{section2}.pth")
    if save_img == True:
        final.write_image(rf"C:\Users\23675\OneDrive\桌面\ABChallenge\Neuralprophet result\result_{consumo}_{section2}.jpeg")
    if para_show == True:
        fig_param.show()
    if pred_show == True:
        final.show()
    #1
    if save_model == True:
        torch.save(m.model.state_dict( ),rf"C:\Users\23675\OneDrive\桌面\ABChallenge\Neuralprophet result\model_{consumo}_{section2}.pth") #####Saves the model's weights and state in the file model.pth, which allows the continuous update once it is deployed on the website.
    return m,pred_train,pred_next,final,fig_param,metrics #pred_train -> train + test


###Noted that the last available data month(2021 December) is not complete, which shows a significant decay in the representation.

## Predicción en el consumo mensual de agua comercial, industrial y doméstico en el municipio de Barcelona.

In [113]:
mensual_Barcelona_comercial = r".\sum_mensual_comercial.xlsx"
mensual_Barcelona_industrial = r".\sum_mensual_industrial.xlsx"
mensual_Barcelona_domestic = r".\sum_mensual_domestic.xlsx"

Enseñar las tendencias del consumo, los cambios de tendencias y las estacionaridades de los datos con el parametro (para_show = True).

In [116]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(mensual_Barcelona_comercial,"BARCELONA",pred_show = False, para_show =True) 

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 87.879% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO 

  0%|          | 0/103 [00:00<?, ?it/s]

comercial
True
sum_mensual_comercial.x


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.00E-01, min: 3.38E-01


  0%|          | 0/103 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.23E-01, min: 2.11E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.07E-01
  0%|          | 0/1000 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[1000/1000]: 100%|██████████| 1000/1000 [00:13<00:00, 75.93it/s, L1Loss=0.0409, MAE=1.38e+6, RMSE=2.61e+6, Loss=0.0296, RegLoss=0, L1Loss_val=0.281, MAE_val=9.5e+6, RMSE_val=1.5e+7]
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corr

In [129]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(mensual_Barcelona_comercial,"BARCELONA",pred_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 87.879% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO 

  0%|          | 0/103 [00:00<?, ?it/s]

comercial
True
sum_mensual_comercial.x


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.84E-01, min: 1.40E+00


  0%|          | 0/103 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.38E-01, min: 1.72E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.84E-01
  0%|          | 0/1000 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[1000/1000]: 100%|██████████| 1000/1000 [00:12<00:00, 82.81it/s, L1Loss=0.041, MAE=1.39e+6, RMSE=2.61e+6, Loss=0.0303, RegLoss=0, L1Loss_val=0.278, MAE_val=9.4e+6, RMSE_val=1.51e+7]
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corr

In [130]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(mensual_Barcelona_industrial,"BARCELONA",pred_show = True,para_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 87.879% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO 

  0%|          | 0/103 [00:00<?, ?it/s]

industrial
True
sum_mensual_industrial.x


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 2.25E-01, min: 6.22E-01


  0%|          | 0/103 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.62E-02, min: 5.44E-02
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.35E-02
  0%|          | 0/1000 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[1000/1000]: 100%|██████████| 1000/1000 [00:16<00:00, 59.08it/s, L1Loss=0.0587, MAE=4.06e+6, RMSE=7.48e+6, Loss=0.0448, RegLoss=0, L1Loss_val=0.441, MAE_val=3.05e+7, RMSE_val=4.65e+7]
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS co

In [131]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(mensual_Barcelona_domestic,"BARCELONA",pred_show = True,para_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 87.879% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO 

  0%|          | 0/103 [00:00<?, ?it/s]

domestic
True
sum_mensual_domestic.x


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.50E-01, min: 2.76E-01


  0%|          | 0/103 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.66E-02, min: 2.76E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.00E-01
  0%|          | 0/1000 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[1000/1000]: 100%|██████████| 1000/1000 [00:09<00:00, 101.27it/s, L1Loss=0.0345, MAE=2.03e+6, RMSE=3.76e+6, Loss=0.0258, RegLoss=0, L1Loss_val=0.315, MAE_val=1.86e+7, RMSE_val=2.86e+7]
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 88.889% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS c

## Predicción de los consumos diarios sobre el municipio de Barcelona

In [132]:
diario_Barcelona_comercial = r"sum_diario_comercial.xlsx"
diario_Barcelona_industrial = r"sum_diario_industrial.xlsx"
diario_Barcelona_domestic = r"sum_diario_domestic.xlsx"

In [133]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(diario_Barcelona_comercial,"BARCELONA",pred_show = True,para_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.904% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 191


comercial
False
sum_diario_comercial.x
--------------------------------


  0%|          | 0/126 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.79E-02, min: 8.17E-01


  0%|          | 0/126 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.02E-02, min: 1.56E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 8.17E-02
  0%|          | 0/191 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[191/191]: 100%|██████████| 191/191 [00:12<00:00, 14.90it/s, L1Loss=0.07, MAE=1.04e+5, RMSE=1.59e+5, Loss=0.0517, RegLoss=0, L1Loss_val=0.233, MAE_val=3.45e+5, RMSE_val=4.63e+5]  
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D correspo

In [134]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(diario_Barcelona_industrial,"BARCELONA",pred_show = True,para_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.904% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 191


  0%|          | 0/126 [00:00<?, ?it/s]

industrial
False
sum_diario_industrial.x
--------------------------------


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.32E-01, min: 1.34E+00


  0%|          | 0/126 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.32E-01, min: 1.58E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.20E-01
  0%|          | 0/191 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[191/191]: 100%|██████████| 191/191 [00:12<00:00, 15.10it/s, L1Loss=0.0615, MAE=2.41e+5, RMSE=3.67e+5, Loss=0.0451, RegLoss=0, L1Loss_val=0.17, MAE_val=6.68e+5, RMSE_val=7.96e+5] 
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D correspo

In [135]:
model,pred_train,pred_next,fig_final,fig_param,metrics = NProphet(diario_Barcelona_domestic,"BARCELONA",pred_show = True,para_show = True)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.904% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 191


  0%|          | 0/126 [00:00<?, ?it/s]

domestic
False
sum_diario_domestic.x
--------------------------------


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.92E-01, min: 1.32E-01


  0%|          | 0/126 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.75E-02, min: 1.32E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.58E-01
  0%|          | 0/191 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - c:\Users\23675\anaconda3\envs\mytorch\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Epoch[191/191]: 100%|██████████| 191/191 [00:11<00:00, 16.04it/s, L1Loss=0.042, MAE=9.64e+4, RMSE=1.59e+5, Loss=0.0327, RegLoss=0, L1Loss_val=0.0355, MAE_val=8.14e+4, RMSE_val=1.26e+5] 
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.907% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresp

Análisis: Gracias a las gráficas estacionarias, podemos observar la similitud en estacionalidad tanto en el consumo doméstico del municipio de Barcelona como en el consumo comercial. De hecho ambos experimentan una bajada durante las vacaciones del verano: julio - septiembre y se recupera a la normalidad con casi igual velocidad que la bajada durante los meses del septiembre - octubre. Gracias a esto, el modelo puede aprender las repeticiones y presentar con una predicción remediada 

## Por el límite del tiempo de ejecución. No hemos podido implementar el modelo a todas las zonas en este notebook. Alternativamente Hemos creado una pagina web para este propósito sin necesidad de correr el modelo nuevamente. Si necesita más información sobre la estadística de tendencia, de estacionaridad, así como las predicciones del consumo de cada municipio en el rango de 4 días o meses. Por favor consulte: https://junjielyu13.github.io/AiguaDeBarcelona_DataChallenge/

### Conclusiones sobre el análisis de datos


Después de analizar los datos, se detecta que en general existen relaciones fuertes entre el número de contadores y el consumo de agua, sobretodo, en el doméstico y comercial, es decir, cuando más contadores tengan instaladas un municipio, más cantidad de agua consumida será registrada. Sin embargo, según las gráficas, y la tabla de correlaciones, no se representa una asociación tan fuerte entre el número de contadores y el consumo industrial excepto en las zonas con un incremento brutal de contadores en determinado periodo tales como Badalona, Cerdanyola de Vallès y Cornellà de Llobregat, los cuales son también municipios con mayor porporción de contadores anormales.

Una posible causa puede ser que las industrias no presentan una demanda alta en la instalación de contadores que los hogares y tiendas. 
A demás , se deduce que existe la posibilidad de un sistema de reciclaje de agua en las industrias, el cual controla y regula la cantidad de agua consumida para evitar el costo generado por el tratamiento de agua residual. Por lo tanto, no se nota una variación grande en el consumo industrial.

En cuanto al hábito de consumir aguas, los resultados indican el periodo de vacaciones (Agosto, Diciembre y Enero) y turismo como factores importantes en la varianción de consumos.


## Porqué queremos aplicar la IA para este proyecto?
1. Predecir el coste futuro para la empresa y mejorar la distribución de agua mediante la observaciónn de tendencias y prediciones através de los contadores de cada zona.
2. A diferencia de los métodos tradicionales estadísticos, este permite a la empresa predecir automáticamente de acuerdo con las tendencias implícitas de los datos. Por ejemplo: el número de contador, eventos, festivos, vacaciones, así como el turismo.

### Surgerencia
Según los datos de la Autoridad del Agua, hemos comprobado que la participación de las aguas subterráneas en el consumo de agua en Cataluña aumenta año tras año. 

Al mismo tiempo, una investigación de los datos experimentales pertinentes muestra que las aguas subterráneas de los alrededores de Barcelona tienen altos niveles de cloruro. 
Tras excluir los resultados de la producción industrial, creemos que esto es consecuencia del descenso de los niveles de las aguas subterráneas debido a la sobreexplotación de las mismas y al contacto directo del agua salada del mar con el agua dulce de los acuíferos costeros a través de la interfase. 

Las agencias de aguas costeras deberían adoptar una estrategia más agresiva para detener esta intrusión, inyectando miles de metros cúbicos de agua regenerada y depurada en las profundidades del acuífero para evitar que esta contaminación siga produciéndose.